In [ ]:
# Import necessary libraries
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from PyPDF2 import PdfReader

embedding_model = "sentence-transformers/all-MiniLM-L6-v2"

# Initialize embeddings using HuggingFace model
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

# Convert the content of the PDF file to a BytesIO stream
file = open('./train_files/academic-policies-and-procedures.pdf', 'rb')
# text_stream = BytesIO(file)
print(file)
# Create a PdfReader object from the stream to extract text 
pdf = PdfReader(file)  
pdf_text = ""
# # Iterate through each page in the PDF and extract text
for page in pdf.pages:
    pdf_text += page.extract_text()  # Concatenate the text from each page


# # ### Create embeddings for the uploaded documents and store in vector store
# # # Initialize a text splitter for processing long texts
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, 
                                                chunk_overlap=10)
# pdf_text = """"""
# # Create documents by splitting the provided texts
documents = text_splitter.create_documents([pdf_text])
# # Create a Faiss index from the embeddings
faiss_index = FAISS.from_documents(documents, embeddings)

indexpath = "data/vectorstore/"

# # Save the Faiss index locally
faiss_index_path = indexpath + 'temp-index'
faiss_index.save_local(faiss_index_path)
